In [4]:
import json
from tqdm import tqdm
import os

imgpath = '/home/cqjtu/Documents/dataset/gligen/test/img'
jsonfile_name = '/home/cqjtu/Documents/dataset/gligen/test/test_caption.json'
jsonfile_origin = '/home/cqjtu/Documents/dataset/gligen/nusenes_train_images_cation.json'
data = dict()

for home, dirs, files in os.walk(imgpath):
    for filename in tqdm(files, desc='generating in tst'):
        with open(jsonfile_origin, 'r') as fo:
            json_data = json.load(fo)
            data[filename] = json_data[filename]

with open(jsonfile_name, 'w') as f:
    json.dump(data, f)

generating in tst: 100%|██████████| 40/40 [00:00<00:00, 230.83it/s]

<module 'json' from '/home/cqjtu/anaconda3/envs/gligen/lib/python3.8/json/__init__.py'>


## data process

In [10]:

import argparse
import torch
from omegaconf import OmegaConf
import numpy as np
import random
from trainer import Trainer
from distributed import synchronize
import os

parser = argparse.ArgumentParser()
parser.add_argument("--DATA_ROOT", type=str,  default="/home/cqjtu/Documents/dataset/gligen/test", help="path to DATA")
parser.add_argument("--OUTPUT_ROOT", type=str,  default="OUTPUT", help="path to OUTPUT")

parser.add_argument("--name", type=str,  default="test", help="experiment will be stored in OUTPUT_ROOT/name")
parser.add_argument("--seed", type=int,  default=123, help="used in sampler")
parser.add_argument("--local_rank", type=int, default=0)
parser.add_argument("--yaml_file", type=str,  default="configs/depth_sem.yaml", help="paths to base configs.")


parser.add_argument("--base_learning_rate", type=float,  default=5e-5, help="")
parser.add_argument("--weight_decay", type=float,  default=0.0, help="")
parser.add_argument("--warmup_steps", type=int,  default=10000, help="")
parser.add_argument("--scheduler_type", type=str,  default='constant', help="cosine or constant")
parser.add_argument("--batch_size", type=int,  default=2, help="")
parser.add_argument("--workers", type=int,  default=1, help="")
parser.add_argument("--official_ckpt_name", type=str,  default="sd-v1-4.ckpt", help="SD ckpt name and it is expected in DATA_ROOT, thus DATA_ROOT/official_ckpt_name must exists")
parser.add_argument("--ckpt", type=lambda x:x if type(x) == str and x.lower() != "none" else None,  default=None,
    help=("If given, then it will start training from this ckpt"
          "It has higher prioty than official_ckpt_name, but lower than the ckpt found in autoresuming (see trainer.py) "
          "It must be given if inpaint_mode is true")
)

parser.add_argument('--inpaint_mode', default=False, type=lambda x:x.lower() == "true", help="Train a GLIGEN model in inpaitning setting")
parser.add_argument('--randomize_fg_mask', default=False, type=lambda x:x.lower() == "true", help="Only used if inpaint_mode is true. If true, 0.5 chance that fg mask will not be a box but a random mask. See code for details")
parser.add_argument('--random_add_bg_mask', default=False, type=lambda x:x.lower() == "true", help="Only used if inpaint_mode is true. If true, 0.5 chance add arbitrary mask for the whole image. See code for details")

parser.add_argument('--enable_ema', default=False, type=lambda x:x.lower() == "true")
parser.add_argument("--ema_rate", type=float,  default=0.9999, help="")
parser.add_argument("--total_iters", type=int,  default=500000, help="")
parser.add_argument("--save_every_iters", type=int,  default=5000, help="")
parser.add_argument("--disable_inference_in_training", type=lambda x:x.lower() == "true",  default=False, help="Do not do inference, thus it is faster to run first a few iters. It may be useful for debugging ")


args = parser.parse_args()
assert args.scheduler_type in ['cosine', 'constant']




n_gpu = int(os.environ["WORLD_SIZE"]) if "WORLD_SIZE" in os.environ else 1
args.distributed = n_gpu > 1

if args.distributed:
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend="nccl", init_method="env://")
    synchronize()



config = OmegaConf.load(args.yaml_file)
config.update( vars(args) )
config.total_batch_size = config.batch_size * n_gpu
if args.inpaint_mode:
    config.model.params.inpaint_mode = True




In [12]:
from dataset.concat_dataset import ConCatDataset
dataset_train=ConCatDataset(config.train_dataset_names,'')

In [13]:
config